In [2]:
import pandas as pd
from scipy.spatial.distance import cosine

In [28]:
data = pd.read_csv('movie_info.csv').iloc[:, range(1,13)]

In [35]:
data.head(n = 10)

#data.iloc[:,[0,3]]

,movidID,title,price,publisher,category,actor,audience_rating,brand,director,feature,genre,release_date
0,310263662,The Passion of the Christ,10.14,Zondervan,DVD,NaN,NaN,Provident Distribution Group,NaN,NaN,Performing Arts,8/31/04
1,767002652,Upstairs Downstairs - The Premiere Season [VHS],9.99,A&E Home Video,Video,Pauline Collins,NR (Not Rated),NaN,Bill Bain,NaN,NaN,1/15/98
2,076780192X,Close Encounters of the Third Kind (Widescreen...,8.49,Sony Pictures Home E,Video,Richard Dreyfuss,PG (Parental Guidance Suggested),NaN,Steven Spielberg,NaN,NaN,8/29/00
3,767802470,Das Boot - The Director's Cut,9.99,Sony Pictures Home Entertainment,DVD,Herbert Gronemeyer,R (Restricted),Columbia/Tristar Studios,Wolfgang Petersen,Factory sealed DVD,Drama,12/9/97
4,767802519,Das Boot - The Director's Cut,9.99,Sony Pictures Home Entertainment,DVD,Herbert Gronemeyer,R (Restricted),Columbia/Tristar Studios,Wolfgang Petersen,Factory sealed DVD,Drama,12/9/97
5,767802624,Men in Black (Collector's Series),1.98,Sony Pictures Home Entertainment,DVD,Linda Fiorentino,PG-13 (Parents Strongly Cautioned),Sony,Barry Sonnenfeld,NaN,NaN,1/1/02
6,767802659,Men in Black (Collector's Series),1.98,Sony Pictures Home Entertainment,DVD,Linda Fiorentino,PG-13 (Parents Strongly Cautioned),Sony,Barry Sonnenfeld,NaN,NaN,1/1/02
7,767805267,Fifth Element [VHS],4.75,Sony Pictures,Video,Bruce Willis,PG-13 (Parents Strongly Cautioned),Columbia Pictures,Luc Besson,NaN,NaN,4/28/98
8,767811100,Fifth Element [VHS],4.75,Sony Pictures,Video,Bruce Willis,PG-13 (Parents Strongly Cautioned),Columbia Pictures,Luc Besson,NaN,NaN,4/28/98
9,767824571,Monty Python and the Holy Grail,14.10,Sony Pictures,DVD,Graham Chapman,PG (Parental Guidance Suggested),NaN,Terry Gilliam,NaN,NaN,9/7/99


In [34]:
# to find out number of nulls
data.isnull().sum()

movidID              0
title                0
price               19
publisher           45
category             0
actor               70
audience_rating     44
brand              266
director            96
feature            296
genre              278
release_date        65
dtype: int64

In [45]:
from random import randint
data['genre'].unique()

array(['Performing Arts', 'Kids & Family', 'Drama', 'Science Fiction',
       'Music Videos & Concerts', 'Musicals', 'Children', 'Horror',
       'Action & Adventure', 'Anime & Manga', 'Comedy', 'Film', 'Romance',
       'Hero'], dtype=object)

In [50]:
len(data['genre'].unique())

14

In [51]:
# since there are quite many missing values in the dataset, and they are categorical variables
# instead of imputing with the mode, which may cause bias
# impute the dataset with random categories present in the column
data = data.apply(lambda x:x.fillna(x.unique()[randint(0, len(x.unique()))]))

In [53]:
data.head(n =5)

,movidID,title,price,publisher,category,actor,audience_rating,brand,director,feature,genre,release_date
0,310263662,The Passion of the Christ,10.14,Zondervan,DVD,Michael Douglas,R (Restricted),Provident Distribution Group,Rob Reiner,COMEDY,Performing Arts,8/31/04
1,767002652,Upstairs Downstairs - The Premiere Season [VHS],9.99,A&E Home Video,Video,Pauline Collins,NR (Not Rated),Uni,Bill Bain,COMEDY,Kids & Family,1/15/98
2,076780192X,Close Encounters of the Third Kind (Widescreen...,8.49,Sony Pictures Home E,Video,Richard Dreyfuss,PG (Parental Guidance Suggested),Uni,Steven Spielberg,COMEDY,Kids & Family,8/29/00
3,767802470,Das Boot - The Director's Cut,9.99,Sony Pictures Home Entertainment,DVD,Herbert Gronemeyer,R (Restricted),Columbia/Tristar Studios,Wolfgang Petersen,Factory sealed DVD,Drama,12/9/97
4,767802519,Das Boot - The Director's Cut,9.99,Sony Pictures Home Entertainment,DVD,Herbert Gronemeyer,R (Restricted),Columbia/Tristar Studios,Wolfgang Petersen,Factory sealed DVD,Drama,12/9/97


In [58]:
# to find out number of nulls again
print "######## Number of missing values in each column #######"
print data.isnull().sum()
print "####### Number of unique values in each column ######"
print data.apply(lambda x:len(x.unique()))

######## Number of missing values in each column #######
movidID            0
title              0
price              0
publisher          0
category           0
actor              0
audience_rating    0
brand              0
director           0
feature            0
genre              0
release_date       0
dtype: int64
####### Number of unique values in each column ######
movidID            449
title               68
price               59
publisher           27
category             2
actor               57
audience_rating      7
brand               22
director            52
feature             19
genre               14
release_date        57
dtype: int64


To compare the similarity between columns, I will use cosine similarity measure between each movies. This means that the data in each column has to be numeric. As such, I used the LabelEncoder package in sklearn to process the data. Note that this might inadvertedly cause the categorical variables to be "ordinal", i.e. existence of some ranks but this should not impact our similarity comparison. The "ordinal" effect of variables will only be more profound when we are trying to predict the outcome with models like Logistic Regression.

In [109]:
from sklearn import preprocessing

encodedCol = data.iloc[:, range(3,12)].apply(preprocessing.LabelEncoder().fit_transform)
encodedCol.head()

,publisher,category,actor,audience_rating,brand,director,feature,genre,release_date
0,26,0,35,5,13,37,6,11,50
1,1,1,41,1,17,5,6,8,3
2,11,1,42,2,17,45,6,8,48
3,12,0,15,5,6,50,11,4,23
4,12,0,15,5,6,50,11,4,23


In [112]:
data.head()

,movidID,title,price,publisher,category,actor,audience_rating,brand,director,feature,genre,release_date
0,310263662,The Passion of the Christ,10.14,Zondervan,DVD,Michael Douglas,R (Restricted),Provident Distribution Group,Rob Reiner,COMEDY,Performing Arts,8/31/04
1,767002652,Upstairs Downstairs - The Premiere Season [VHS],9.99,A&E Home Video,Video,Pauline Collins,NR (Not Rated),Uni,Bill Bain,COMEDY,Kids & Family,1/15/98
2,076780192X,Close Encounters of the Third Kind (Widescreen...,8.49,Sony Pictures Home E,Video,Richard Dreyfuss,PG (Parental Guidance Suggested),Uni,Steven Spielberg,COMEDY,Kids & Family,8/29/00
3,767802470,Das Boot - The Director's Cut,9.99,Sony Pictures Home Entertainment,DVD,Herbert Gronemeyer,R (Restricted),Columbia/Tristar Studios,Wolfgang Petersen,Factory sealed DVD,Drama,12/9/97
4,767802519,Das Boot - The Director's Cut,9.99,Sony Pictures Home Entertainment,DVD,Herbert Gronemeyer,R (Restricted),Columbia/Tristar Studios,Wolfgang Petersen,Factory sealed DVD,Drama,12/9/97


In [126]:
len(data['title'].unique())

68

This raises an alarm where it seems like we only have got 68 unique movies in this dataset, based on the names of the movies. However, they have unique movieID as extracted from the database. 

We will first proceed with finding out the similarity matrix for the 449 movies (asssuming they are unique). If time permits, we will revisit this problem and learn to extract unique movies from Amazon API.

In [114]:
df_part1 = data[['movidID', 'price']]
df_part1.head()
# goal: join df_part1 with encoded Col

,movidID,price
0,310263662,10.14
1,767002652,9.99
2,076780192X,8.49
3,767802470,9.99
4,767802519,9.99


In [115]:
joinedDF = pd.concat([df_part1, encodedCol], axis=1)
joinedDF.head()

,movidID,price,publisher,category,actor,audience_rating,brand,director,feature,genre,release_date
0,310263662,10.14,26,0,35,5,13,37,6,11,50
1,767002652,9.99,1,1,41,1,17,5,6,8,3
2,076780192X,8.49,11,1,42,2,17,45,6,8,48
3,767802470,9.99,12,0,15,5,6,50,11,4,23
4,767802519,9.99,12,0,15,5,6,50,11,4,23


In [116]:
# think about what is good categorical variables 
# that would be useful in the similarity matrix between movies
df = joinedDF.T
df.head(n = 10)

,0,1,2,3,4,5,6,7,8,9,...,439,440,441,442,443,444,445,446,447,448
movidID,310263662,767002652,076780192X,767802470,767802519,767802624,767802659,767805267,767811100,767824571,...,B00006RCNW,B00007E2F5,B000083C6V,B00008DDVU,B00008DDXB,B00008EY63,B00008W64E,B00008YGRU,B00009MGEM,B00009OOFA
price,10.14,9.99,8.49,9.99,9.99,1.98,1.98,4.75,4.75,14.1,...,8.15,8.15,8.15,8.15,8.15,4.81,4.81,14.23,14.23,14.23
publisher,26,1,11,12,12,12,12,10,10,10,...,7,7,7,7,7,10,10,5,5,5
category,0,1,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,1,0,0,0
actor,35,41,42,15,15,32,32,5,5,13,...,12,12,12,12,12,47,47,30,30,30
audience_rating,5,1,2,5,5,4,4,4,4,2,...,5,5,5,5,5,4,4,6,6,6
brand,13,17,17,6,6,14,14,5,5,17,...,15,15,15,15,15,17,17,0,0,0
director,37,5,45,50,50,4,4,29,29,47,...,49,49,49,49,49,16,16,37,37,37
feature,6,6,6,11,11,6,6,6,6,6,...,11,11,11,11,11,6,6,1,1,1
genre,11,8,8,4,4,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [127]:
# set column names as the movieID
# remove title as it is not a good measure of similarity with other movies (perhaps could use count vectorizer if time allows)
df.columns = list(df.loc['movidID'].values)

In [128]:
df.head()

,310263662,767002652,076780192X,767802470,767802519,767802624,767802659,767805267,767811100,767824571,...,B00006RCNW,B00007E2F5,B000083C6V,B00008DDVU,B00008DDXB,B00008EY63,B00008W64E,B00008YGRU,B00009MGEM,B00009OOFA
movidID,310263662,767002652,076780192X,767802470,767802519,767802624,767802659,767805267,767811100,767824571,...,B00006RCNW,B00007E2F5,B000083C6V,B00008DDVU,B00008DDXB,B00008EY63,B00008W64E,B00008YGRU,B00009MGEM,B00009OOFA
price,10.14,9.99,8.49,9.99,9.99,1.98,1.98,4.75,4.75,14.1,...,8.15,8.15,8.15,8.15,8.15,4.81,4.81,14.23,14.23,14.23
publisher,26,1,11,12,12,12,12,10,10,10,...,7,7,7,7,7,10,10,5,5,5
category,0,1,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,1,0,0,0
actor,35,41,42,15,15,32,32,5,5,13,...,12,12,12,12,12,47,47,30,30,30


In [129]:
# remove movidID and title as they are not representative of how similar the movies are to each other
# movie name could be treated with count vectorizer instead
dfnew = df.drop('movidID')
dfnew.head()

,310263662,767002652,076780192X,767802470,767802519,767802624,767802659,767805267,767811100,767824571,...,B00006RCNW,B00007E2F5,B000083C6V,B00008DDVU,B00008DDXB,B00008EY63,B00008W64E,B00008YGRU,B00009MGEM,B00009OOFA
price,10.14,9.99,8.49,9.99,9.99,1.98,1.98,4.75,4.75,14.1,...,8.15,8.15,8.15,8.15,8.15,4.81,4.81,14.23,14.23,14.23
publisher,26,1,11,12,12,12,12,10,10,10,...,7,7,7,7,7,10,10,5,5,5
category,0,1,1,0,0,0,0,1,1,0,...,0,0,0,0,0,1,1,0,0,0
actor,35,41,42,15,15,32,32,5,5,13,...,12,12,12,12,12,47,47,30,30,30
audience_rating,5,1,2,5,5,4,4,4,4,2,...,5,5,5,5,5,4,4,6,6,6


In [130]:
dfnew.shape

(10, 449)

In [131]:
# initialize a movie similarity matrix
movie_similarity_matrix = pd.DataFrame(index = dfnew.columns, columns = dfnew.columns)
movie_similarity_matrix.head()

,310263662,767002652,076780192X,767802470,767802519,767802624,767802659,767805267,767811100,767824571,...,B00006RCNW,B00007E2F5,B000083C6V,B00008DDVU,B00008DDXB,B00008EY63,B00008W64E,B00008YGRU,B00009MGEM,B00009OOFA
310263662,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767002652,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
076780192X,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767802470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
767802519,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We will use Cosine similarity to measure the similarity between two non-zero vectors. Two vectors of the the same orientation will have a cosine similarity of 1, while two vectors of  $90^{\circ}$  will have a cosine similarity of 0.

In [132]:
# generate the similarity matrix
# Lets fill in those empty spaces with cosine similarities
#outer loop to loop through each item
for i in range(0,len(movie_similarity_matrix.columns)) :
    #inner loop to identify similarity of the other columns with the column
    for j in range(0, len(movie_similarity_matrix.columns)) :
    # Fill in placeholder with cosine similarities
    # identical items will have value of 1
      movie_similarity_matrix.ix[i,j] = 1 - cosine(dfnew.ix[:,i], dfnew.ix[:,j])

In [133]:
movie_similarity_matrix.head()

,310263662,767002652,076780192X,767802470,767802519,767802624,767802659,767805267,767811100,767824571,...,B00006RCNW,B00007E2F5,B000083C6V,B00008DDVU,B00008DDXB,B00008EY63,B00008W64E,B00008YGRU,B00009MGEM,B00009OOFA
310263662,1,0.606388,0.97118,0.865568,0.865568,0.630393,0.630393,0.918552,0.918552,0.925998,...,0.70835,0.70835,0.70835,0.70835,0.70835,0.856603,0.856603,0.941133,0.941133,0.941133
767002652,0.606388,1,0.671658,0.437978,0.437978,0.937578,0.937578,0.311376,0.311376,0.40497,...,0.468883,0.468883,0.468883,0.468883,0.468883,0.916011,0.916011,0.511903,0.511903,0.511903
076780192X,0.97118,0.671658,1,0.887715,0.887715,0.637779,0.637779,0.893322,0.893322,0.929197,...,0.764455,0.764455,0.764455,0.764455,0.764455,0.887384,0.887384,0.950118,0.950118,0.950118
767802470,0.865568,0.437978,0.887715,1,1,0.444173,0.444173,0.902115,0.902115,0.905265,...,0.935945,0.935945,0.935945,0.935945,0.935945,0.666852,0.666852,0.842423,0.842423,0.842423
767802519,0.865568,0.437978,0.887715,1,1,0.444173,0.444173,0.902115,0.902115,0.905265,...,0.935945,0.935945,0.935945,0.935945,0.935945,0.666852,0.666852,0.842423,0.842423,0.842423


In [134]:
# sanity check
movie_similarity_matrix.iloc[:, 1].sort_values(ascending = False)

767002652            1
1415724784    0.959727
080017948X    0.959727
1404983082    0.959727
1415718113    0.959727
1415724806    0.959727
1417030321    0.959727
1417054069    0.959727
1400322715    0.959727
6304089767    0.953092
6304178352    0.953092
6304214502    0.953092
6303965415    0.953092
6303961592    0.953092
6303921248    0.953092
6303908306    0.953092
630365147X    0.953092
6304176287    0.953092
6304117752    0.953092
767802659     0.937578
767802624     0.937578
6302637899    0.926992
6303122647    0.926992
6302760046    0.926992
6302787068    0.926992
6302794331    0.926992
6302822696    0.926992
6302872766    0.926992
6302921058    0.926992
6302968143    0.926992
                ...   
6300214087    0.369554
783225970     0.343223
078322687X    0.343223
783227434     0.343223
783227507     0.343223
783228015     0.343223
6303187064    0.326958
6303212263    0.326958
6303212379    0.326958
767811100     0.311376
767805267     0.311376
B00005JPNY    0.289964
B00005K3OT 

In [135]:
# initialize empty dataframe to store the top 10 closest neighbors to each item
data_neighbors = pd.DataFrame(index = movie_similarity_matrix.columns, columns = range(1, 10))

In [136]:
# loop through similarity matrix above and fill in the neighbor names
# shortlist the top 5 most similar hours to respective hours
for i in range(0, len(movie_similarity_matrix.columns)):
    data_neighbors.iloc[i, :] = movie_similarity_matrix.iloc[:, i].sort_values(ascending = False)[1:10].index
    
data_neighbors.head()

,1,2,3,4,5,6,7,8,9
310263662,790731479,790729733,790729989,790730960,788812408,788812807,788814664,783222734,076780192X
767002652,1415724784,080017948X,1404983082,1415718113,1415724806,1417030321,1417054069,1400322715,6304089767
076780192X,B00005JL3A,B00005JKQZ,B00005JKVZ,B00005JKZY,B00005JL3T,792148061,792165020,792164903,792158288
767802470,767802519,B00005JLBQ,B00005JLKN,B00005JL78,B00005JL8F,B00005JLF2,B00005JLET,B00005JLBE,B00005JO1R
767802519,767802519,B00005JLBQ,B00005JLKN,B00005JL78,B00005JL8F,B00005JLF2,B00005JLET,B00005JLBE,B00005JO1R
